In [1]:
from notus import Encoder, Decoder, utils, FileDataset, Muon, ByteLevelTokenizer, eval, CompressionEngine, FileFormer
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint

In [2]:

#tokenizer
tokenizer = ByteLevelTokenizer()

#configs
configs = utils.load_config("/config/config.yml")

In [3]:

#models
encoder = Encoder(**configs['encoder'])
decoder = Decoder(**configs['decoder'])

#loss
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=1)

In [5]:
fileformer = FileFormer.load_from_checkpoint("/Users/daniilogorodnikov/model-epoch=00-step=20000.ckpt", encoder=encoder, decoder=decoder, loss_fn=loss_fn, config=configs, device="mps")

/Users/daniilogorodnikov/anaconda3/envs/notus/lib/python3.12/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.5.5, which is newer than your current Lightning version: v2.5.0.post0


In [6]:
from pathlib import Path
import hashlib
from notus.tokenizer.utils import get_file_ext_as_token
import mmap

In [53]:
file_path = Path("../notus/engine/lorem_ipsum.txt")

In [54]:
sha256_hash = hashlib.sha256()
with open(file_path, 'rb') as f:
    chunk = f.read(4096)
    print(hashlib.sha256(chunk).hexdigest())

e983b3dab739d02999a9d4089820e29dfd1030f640e8f5ce5c8ebffcd9719bc7


In [78]:
eng = CompressionEngine({"rule":311, "chunk_size":4096}, fileformer)

In [79]:
out = eng.compress(chunk)

In [80]:
iiii = tokenizer.encode(chunk.hex())

In [81]:
out

{'hash': 'e983b3dab739d02999a9d4089820e29dfd1030f640e8f5ce5c8ebffcd9719bc7',
 'data': '4c6f726d206973756d646f6c72207374206165742c636f6e6563747475726164696973636e67206c697420736520646f6569756d6f6474656d6f72206e6369696475742075206c616f72656574206f6c6f65206d676e61616c6975612e5574206e696d616420696e6920766569616d20717573206e7374726420656572637461746f6e206c6c61636f2061626f6973206973697574206c697169702078206520636f6d6f6420636f73657161742e447569206175652069757265646f6c7220692072657265686e64656974206e20766c757061746576656c742065736520696c6c6d20646c6f72206575667567617420756c6c207061696174722e2078636574657520736974206f636165617420757069617461206e6f207072696465742c20756e74696e20756c70207175206f6669636920646565727574206d6c6c6920616e6d2069206573206c616f72752e204c72656d6970736d20646c6f72736974616d652c20636e736574657472206169706963696e20656c742c206564206f206575736d6420746d706f20696e696469756e7475742061626f65206520646f6f72656d61676120616971752e205520656e6d2061206d69696d20656e696d2c207569736e6f737275646

In [82]:
ttt = eng.decompress("e983b3dab739d02999a9d4089820e29dfd1030f640e8f5ce5c8ebffcd9719bc7", out['data'], [184])

In [85]:
ttt.shape

torch.Size([1, 4096, 258])

In [86]:
fff = torch.argmax(ttt, dim=2)

In [87]:
hex_out = tokenizer.decode(fff.tolist()[0])

In [88]:
hex_out

'4c6f72b96d20693173756db9646f6c2072207368742061b965742cb9636f6eb9656374207475722061646931697363206e6720206c6974202073653120646f20656975206d6f642074656d206f7220206e6369316964753174207520206c61f76f726520657420f76f6c6f2065206d20676e6120616c692075612e20557420206e696d20616420f7696e69202076653169616d202071752073206e20737472206420652065726320746174316f6e20206c6c61b9636f202061626f316973203169736931757420206c697131697020b97820653620636f366d6f642020636f687365713661742e2044756965206175206520692075726520646f6c207220693120726520726568316e646520697420206e2076366c7570206174652076656c207420652073652020696c6c206d2064206c6f7220206575206675672061742020756c6c202070612069617420722e202078636520746575202073692074206f2063616520617420207570693161746120206e6f202070722069646520742c2020756e7420696e2020756c702020717520206f662069636920206465206572753174206d206c6c693120616eb96d20692020657331206c61206f7275202e204c2072656d20697073b96d2064206c6f722073697420616d65202c2063206e73653174657420722061206970692063696e2020656c3

In [89]:
out['data']

'4c6f726d206973756d646f6c72207374206165742c636f6e6563747475726164696973636e67206c697420736520646f6569756d6f6474656d6f72206e6369696475742075206c616f72656574206f6c6f65206d676e61616c6975612e5574206e696d616420696e6920766569616d20717573206e7374726420656572637461746f6e206c6c61636f2061626f6973206973697574206c697169702078206520636f6d6f6420636f73657161742e447569206175652069757265646f6c7220692072657265686e64656974206e20766c757061746576656c742065736520696c6c6d20646c6f72206575667567617420756c6c207061696174722e2078636574657520736974206f636165617420757069617461206e6f207072696465742c20756e74696e20756c70207175206f6669636920646565727574206d6c6c6920616e6d2069206573206c616f72752e204c72656d6970736d20646c6f72736974616d652c20636e736574657472206169706963696e20656c742c206564206f206575736d6420746d706f20696e696469756e7475742061626f65206520646f6f72656d61676120616971752e205520656e6d2061206d69696d20656e696d2c207569736e6f7372756465786563697474696f20756c616d63206c616f7269206e6969207520616c7175692065786561206f6d6d646

In [90]:
def string_similarity(a: str, b: str) -> float:
    # Преобразуем строки в множества символов
    set_a = set(a)
    set_b = set(b)

    # Находим пересечение символов
    common = set_a.intersection(set_b)

    # Вычисляем процент совпадения
    # Берем максимальную длину множества символов для нормализации
    total_chars = max(len(set_a), len(set_b))

    if total_chars == 0:  # Обработка случая пустых строк
        return 0.0

    similarity = (len(common) / total_chars) * 100
    return round(similarity, 2)

In [92]:
string_similarity(chunk.hex(), hex_out)

87.5

In [93]:
tmp = bytearray.fromhex(hex_out)

In [94]:
with open("../notus/engine/tmp.txt", "wb") as binary_file:

    binary_file.write(tmp)

In [52]:
def create_lorem_ipsum_file(target_size=4096, output_file="lorem_ipsum.txt"):
    # Стандартный текст Lorem Ipsum
    lorem = (
        "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor "
        "incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud "
        "exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure "
        "dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. "
        "Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt "
        "mollit anim id est laborum. "
    )

    # Повторяем текст, чтобы превысить целевой размер
    full_text = lorem * (target_size // len(lorem) + 1)
    # Обрезаем до точного размера
    final_text = full_text[:target_size]

    # Записываем в файл
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(final_text)

    # Проверяем размер файла
    file_size = len(final_text.encode("utf-8"))
    print(f"Создан файл {output_file} размером {file_size} байт")

if __name__ == "__main__":
    create_lorem_ipsum_file()

Создан файл lorem_ipsum.txt размером 4096 байт


In [4]:
eval.eva

TypeError: eval expected at least 1 argument, got 0